In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray
import math
import shapely
import xvec

from shapely.geometry import Point, MultiPoint

In [2]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]

# Convert tuples with coordinates to shapely geometry
Should eventually be deprecated by already using shapely geometries
in the first place.

In [3]:
def convert_to_point(row):
    return Point(row)

In [4]:
def convert_to_multipoint(row):
    if row:
        return MultiPoint(row)
    else:
        return None

In [5]:
# Convert start and end point to geometry
tuple_columns = ["start_point", "end_point"]

for col in tuple_columns:
    sightlines[col+"_geom"] = sightlines[col].apply(convert_to_point)


# Convert sight line points to geometry
sightlines["sl_points"] = sightlines["sight_line_points"].apply(convert_to_multipoint)

# Load raster dtm

In [6]:
raster = rioxarray.open_rasterio("data/dtm/dtm_raw/RGEALTI_FXX_1025_6285_MNT_LAMB93_IGN69.asc")
raster = raster.drop_vars("band").squeeze()

# Extract z coordinates from raster

In [7]:
# Extract z coords from raster
z_start = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["start_point_geom"], x_coords="x",y_coords="y")

# Convert do gpd
z_start = z_start.xvec.to_geopandas().rename(columns={0: "z"})

# Append z values to points
z_start["start_point_3d"] = z_start.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
z_start = z_start.drop(columns=["geometry", "z"])



# Extract z coords from raster
z_end = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["end_point_geom"], x_coords="x",y_coords="y")

# Convert do gpd
z_end = z_end.xvec.to_geopandas().rename(columns={0: "z"})

# Append z values to points
z_end["end_point_3d"] = z_end.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
z_end = z_end.drop(columns=["geometry", "z"])

In [8]:
z_points_list = []

for row in sightlines["sl_points"]:
    if row is not None:
        points = row.geoms

        z_points = raster.drop_vars("spatial_ref").xvec.extract_points(points=points, x_coords="x",y_coords="y")
        z_points = z_points.xvec.to_geopandas().rename(columns={0: "z"})

        z_points["geometry"] = z_points.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
        z_points = z_points.drop(columns="z")
        
        multipoint = MultiPoint(z_points["geometry"].tolist())

    else:
        multipoint = None

    z_points_list.append(multipoint)

## Put results together

In [9]:
sightlines = pd.concat([z_start, z_end], axis=1)

sightlines = sightlines.rename(columns={"start_point_3d": "sl_start",
                                          "end_point_3d": "sl_end"})

sightlines["sl_points"] = z_points_list

# Compute slope function
Finally, yaaay.

In [10]:
NODATA_RASTER = raster.rio.nodata

def compute_slope(road_row):
    start = road_row.sl_start   # Point z
    end = road_row.sl_end       # Point z
    slp = road_row.sl_points    # Multipoint z

    if slp == None:
        # Case when there is no sight line point (e.g. when the road is too short)
        # just computes slope between start and end
        if start.z == NODATA_RASTER or end.z == NODATA_RASTER:
            # Case when there is at least one invalid z coord
            return 0, 0, 0, False
        slope_percent = abs(start.z - end.z) / shapely.distance(start, end)
        slope_degree = math.degrees(math.atan(slope_percent))

        return slope_percent, slope_degree, 1, True
    
    # From Multipoint z to Point z list
    slp_list = [p for p in slp.geoms]

    points = []

    points.append(start)
    # From Point z list to all points list
    for p in slp_list:
        points.append(p)
    points.append(end)

    # number of points
    nb_points = len([start]) + len([end]) + len(slp_list)

    # temporary variables to store inter slope values
    sum_slope_percent = 0
    sum_slope_radian = 0
    sum_nb_points = 0

    # if there is one or more sight line points
    for i in range (1, nb_points-1):
        a = points[i-1]
        b = points[i+1]

        if a.z == NODATA_RASTER or b.z == NODATA_RASTER:
            # Case when there is no valid z coord in slpoint
            continue
            
        sum_nb_points += 1
        inter_slope_percent = abs(a.z - b.z) / shapely.distance(a, b)

        sum_slope_percent += inter_slope_percent
        sum_slope_radian += math.atan(inter_slope_percent)

    if sum_nb_points == 0:
        # Case when no slpoint has a valid z coord
        # Unable to compute slope
        return 0, 0, 0, False
        
    # compute mean of inter slopes
    slope_percent = sum_slope_percent/sum_nb_points
    slope_degree = math.degrees(sum_slope_radian/sum_nb_points)

    return slope_degree, slope_percent, sum_nb_points, True

# Compute results

In [11]:
slope_values = []

for _, road_row in sightlines.iterrows():

    # display([p for p in road_row.sl_points.geoms])
    # break

    slope_degree, slope_percent, n_slopes, slope_valid = compute_slope(road_row)

    slope_values.append([slope_degree, slope_percent, n_slopes, slope_valid])

    df_slopes = pd.DataFrame(slope_values,
                             columns=["slope_degree",
                                      "slope_percent",
                                      "n_slopes",
                                      "slope_valid"])

In [12]:
df_slopes

,slope_degree,slope_percent,n_slopes,slope_valid
0,2.305050,0.040300,28,True
1,1.094379,0.019111,33,True
2,3.159987,0.055233,15,True
3,2.404723,0.042030,34,True
4,0.878624,0.015341,13,True
...,...,...,...,...
2162,0.284191,0.004960,7,True
2163,4.664951,0.081627,5,True
2164,9.312918,0.166963,9,True
2165,6.194412,0.109414,31,True


In [14]:
df_slopes.to_parquet("my_results/method_3/d06_dtm_result.parquet")